In [8]:
import matplotlib as plt
import numpy as np
import pandas as pd
import json
import subprocess
from glob2 import glob

In [5]:
import mir_eval as me

In [3]:
dcml_chordtype_map = {
    "M": "major",
    "m": "minor",
    "Mm7": "dominant-7th",
    "o": "diminished",
    "o7": "full-diminished",
    "mm7": "minor-7th",
    "%7": "half-diminished",
    "MM7": "major-7th",
    "+": "augmented",
    "mM7": "minor-major-7th",
    "+7": "augmented-7th",
}

----

## Loading the Results
Loads all json files from the given folder

In [130]:
cols = ['corpus', 'piece', 'algorithm', 'accuracy', 'likelihood', 'chordLabels', 'slices']

In [171]:
def load_dataset(path):
    jsonFiles = glob(path + '/*/*.json') #Can be used absolute or relative paths 
    df = pd.DataFrame(columns=cols) 
    lbls = pd.DataFrame(columns=["piece", "corpus", "labels"])
    for jsonFile in jsonFiles:
        # df = pd.read_json(jsonFile)
        try:
            with open(jsonFile, 'r') as f:
                data = json.loads(f.read())
        except:
            print("Error with {}".format(jsonFile))
            continue

        #lbls.append(data['groundTruth'])
        newLbls = pd.DataFrame([{"corpus":data['corpus'], "piece":data["piece"], "labels": data["groundTruth"]}])
        lbls = pd.concat([lbls, newLbls])

        newdf = pd.json_normalize(data, record_path = ['results'], 
            meta = ['corpus', 'piece'])
        
        #newdf['groundTruth']= data['groundTruth']
        df = pd.concat([df, newdf])
    return (df.set_index(["corpus", "piece"]),lbls.set_index(["corpus", "piece"]))


In [181]:
(results,groundTruth) = load_dataset("outputs")

Error with outputs/ABC/n01op18-1_02.json
Error with outputs/grieg_lyric_pieces/op12n06.json
Error with outputs/grieg_lyric_pieces/op12n07.json
Error with outputs/grieg_lyric_pieces/op38n01.json
Error with outputs/grieg_lyric_pieces/op12n03.json
Error with outputs/grieg_lyric_pieces/op12n04.json
Error with outputs/grieg_lyric_pieces/op12n05.json
Error with outputs/tests/shortt1.json
Error with outputs/schumann_kinderszenen/n07.json
Error with outputs/schumann_kinderszenen/n06.json
Error with outputs/schumann_kinderszenen/n10.json
Error with outputs/schumann_kinderszenen/n01.json
Error with outputs/schumann_kinderszenen/n03.json
Error with outputs/schumann_kinderszenen/n02.json
Error with outputs/schumann_kinderszenen/n08.json


In [163]:
list(results.index.levels[1].unique())

['n01op18-1_01',
 'n01op18-1_03',
 'n01op18-1_04',
 'n02op18-2_01',
 'n02op18-2_02',
 'n02op18-2_03',
 'n02op18-2_04',
 'n04',
 'n05',
 'n09',
 'op12n01',
 'op12n02',
 'op12n08',
 'op38n02',
 'shortest']

In [164]:
results.groupby(["algorithm"]).agg([np.mean, np.std])

/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_63618/607374105.py:1: FutureWarning: ['chordLabels', 'slices'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x.groupby(["algorithm"]).agg([np.mean, np.std])


accuracy              likelihood              
                     mean       std          mean           std
algorithm                                                      
RandomParseSBS   0.568167  0.111581 -1.455901e+06  1.650884e+06
RandomSample     0.027638  0.032723 -8.931068e+06  7.789483e+06
RandomSampleSBS  0.364892  0.107031 -9.922110e+05  1.206095e+06

In [165]:
results.groupby(["corpus", "algorithm"]).agg([np.mean, np.std])

/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_63618/3892531011.py:1: FutureWarning: ['chordLabels', 'slices'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x.groupby(["corpus", "algorithm"]).agg([np.mean, np.std])


accuracy              likelihood  \
                                           mean       std          mean   
corpus                algorithm                                           
ABC                   RandomParseSBS   0.577378  0.052652 -1.252005e+06   
                      RandomSample     0.025685  0.008679 -7.758274e+06   
                      RandomSampleSBS  0.366933  0.032158 -8.635205e+05   
grieg_lyric_pieces    RandomParseSBS   0.516857  0.158688 -2.286973e+06   
                      RandomSample     0.020966  0.020514 -1.309550e+07   
                      RandomSampleSBS  0.290632  0.098249 -1.343494e+06   
schumann_kinderszenen RandomParseSBS   0.540148  0.162034 -1.626033e+06   
                      RandomSample     0.025564  0.020118 -1.091638e+07   
                      RandomSampleSBS  0.383155  0.183352 -1.355029e+06   
tests                 RandomParseSBS   0.750000  0.000000 -1.401125e+01   
                      RandomSample     0.083333  0.144338 -2.232674e+01   
                      RandomSampleSBS  0.583333  0.144338 -1.309518e+01   

                                                     
                                                std  
corpus                algorithm                      
ABC                   RandomParseSBS   9.556629e+05  
                      RandomSample     5.173544e+06  
                      RandomSampleSBS  6.995455e+05  
grieg_lyric_pieces    RandomParseSBS   2.394480e+06  
                      RandomSample     1.296683e+07  
                      RandomSampleSBS  1.492970e+06  
schumann_kinderszenen RandomParseSBS   2.439028e+06  
                      RandomSample     5.346036e+06  
                      RandomSampleSBS  2.151019e+06  
tests                 RandomParseSBS   7.291452e-02  
                      RandomSample     3.600534e+00  
                      RandomSampleSBS  6.000642e-01

# Close Error Analysis

Attributes: chord type, root note, etc

#### Zoom in on ABC: n01op18-1_04

In [166]:
groundTruth.loc['ABC','n01op18-1_04']

labels    [FM, CMm7, CMm7, CM, FM, FM, CM, CMm7, FM, CM,...
Name: (ABC, n01op18-1_04), dtype: object

In [174]:
results.loc['ABC','n01op18-1_04'].iloc[1]

/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_63618/549337838.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  x.loc['ABC','n01op18-1_04'].iloc[1]


algorithm                                         RandomParseSBS
accuracy                                                0.618729
likelihood                                       -1003360.058819
chordLabels    [FM, CMm7, CMm7, GM, FM, FM, CM, CM, FM, CM, C...
slices         [[E6, F6, C6], [E5, C4, B♭4, G3], [E3, C4, B♭4...
Name: (ABC, n01op18-1_04), dtype: object

In [180]:
print (list(groundTruth.loc['ABC','n01op18-1_04'])[0][:15])

['FM', 'CMm7', 'CMm7', 'CM', 'FM', 'FM', 'CM', 'CMm7', 'FM', 'CM', 'CMm7', 'FM', 'CM', 'CMm7', 'FM']


In [179]:
print (list(results.loc['ABC','n01op18-1_04'].iloc[1]["chordLabels"])[:15])

['FM', 'CMm7', 'CMm7', 'GM', 'FM', 'FM', 'CM', 'CM', 'FM', 'CM', 'CM', 'FM', 'CM', 'CMm7', 'FM']


/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_63618/4269611082.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  print (list(x.loc['ABC','n01op18-1_04'].iloc[1]["chordLabels"])[:15])


In [ ]:
me.

## Confusion Matrices

### By Chord Type